In [1]:
import os
import pandas as pd

from wordcloud import WordCloud
import copy
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from kiwipiepy import Kiwi
kiwi = Kiwi(typos='basic')
from kiwipiepy.utils import Stopwords
stopwords = Stopwords()

C:\Users\wooyong\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
# 각 폴더의 경로 설정
folders = [
    'I:\\내 드라이브\\4-1\\텍데분\\크롤링\\크롤링폴더\\국민의힘',
    'I:\\내 드라이브\\4-1\\텍데분\\크롤링\\크롤링폴더\\날씨',
    'I:\\내 드라이브\\4-1\\텍데분\\크롤링\\크롤링폴더\\민주당',
    'I:\\내 드라이브\\4-1\\텍데분\\크롤링\\크롤링폴더\\의대',
    'I:\\내 드라이브\\4-1\\텍데분\\크롤링\\크롤링폴더\\전쟁',
    'I:\\내 드라이브\\4-1\\텍데분\\크롤링\\크롤링폴더\\정부'
]
# 빈 데이터프레임 생성
combined_df = pd.DataFrame()

# 전체 파일 개수
total_file_count = 0

# 각 폴더를 순회
for folder_path in folders:
    if os.path.isdir(folder_path):
        # 폴더 내의 모든 CSV 파일을 확인
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(folder_path, file_name)
                # CSV 파일을 읽어서 데이터프레임에 추가
                df = pd.read_csv(file_path)
                combined_df = pd.concat([combined_df, df], ignore_index=True)
                total_file_count += 1

In [5]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15750 entries, 0 to 15749
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   link     15750 non-null  object
 1   pubdate  15750 non-null  object
 2   title    15750 non-null  object
 3   body     15720 non-null  object
dtypes: object(4)
memory usage: 492.3+ KB


In [6]:
# # 중복 행 제거
combined_df.drop_duplicates(subset=['title'], inplace=True)

In [7]:
# 결과 출력
print(f"총 CSV 파일 개수: {total_file_count}")
print(f"중복 제거 후 행 개수: {combined_df.shape[0]}")

# 중복 제거된 데이터를 새로운 CSV 파일로 저장 (원하는 경우)
output_path = 'G:\\내 드라이브\\4-1\\텍데분\\크롤링\\크롤링폴더\\combined.csv'
# combined_df.to_csv(output_path, index=False)

총 CSV 파일 개수: 192
중복 제거 후 행 개수: 7037


In [8]:
kiwi = Kiwi(typos='basic')
stopwords = Stopwords()

def preprocess_korean(text, analyzer=kiwi, stopwords=stopwords):
    my_text = copy.copy(text)
    my_text = my_text.replace('\n', ' ') # (1) 줄바꿈 문자 제거
    my_text = kiwi.space(my_text) # (2) 띄어쓰기 교정
    sents = kiwi.split_into_sents(my_text) # (3) 문장 토큰화
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
    all_result = []
    for sent in sents:
    
        token_result = kiwi.tokenize(sent.text, stopwords=stopwords) # (4) 형태소 분석 + 간단한 오타 교정
        token_result = kiwi.join(token_result)
        token_result = p.sub(' ', token_result) # (5) 특수 문자 제거 (=한글을 제외한 문자 제거)
        all_result.append(token_result) # (6) 형태소 분석한 결과를 다시 join
    
    all_result = ' '.join(all_result) # (7) 모든 문장을 하나의 string으로 join

    return all_result

# # 예시 텍스트
# example_text = "안녕하세요! 이 문장은 전처리 됩니다.\n이 문장도 전처리됩니다. 1234"
# print(preprocess_korean(example_text))

In [9]:
%%time
combined_df['preprocessed_title'] = combined_df['title'].apply(lambda x: preprocess_korean(x))

CPU times: total: 42.3 s
Wall time: 55.4 s


In [10]:
# # 중복 행 제거
combined_df.drop_duplicates(subset=['preprocessed_title'], inplace=True)

In [11]:
combined_df.shape

(6791, 5)